In [ ]:
install.packages("googledrive")
install.packages("httpuv")
install.packages("R.utils")
install.packages("keras")
install.packages("Metrics")
install.packages("caret")
library("googledrive") 
library("httpuv")
library("R.utils")
library("httr")
library(keras)
library(dplyr)
library(tidyverse)
library(caret)
library(readr)
library(Metrics)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘promises’, ‘later’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘RcppTOML’, ‘here’, ‘png’, ‘config’, ‘tfautograph’, ‘reticulate’, ‘tensorflow’, ‘tfruns’, ‘zeallot’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.1 (2020-08-26 16:20:06 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.24.0 (2020-08-26 16:11:58 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

   

In [ ]:
my_check <- function() {return(TRUE)}
reassignInPackage("is_interactive", pkgName = "httr", my_check) 
options(rlang_interactive=TRUE)

In [ ]:
drive_auth(use_oob = TRUE, cache = FALSE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/auth?client_id=603366585132-dpeg5tt0et3go5of2374d83ifevk5086.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code



Enter authorization code: 4/1AX4XfWgJ8ArbE0ciVYzvSzBdOhyIgemcIb6r1RBc1LbzGRpg92a-_E3XFfU


In [ ]:
getwd()

[1] "/content"

In [ ]:
list.files(path="./sample_data", pattern=NULL, all.files=FALSE,
    full.names=FALSE)

[1] "anscombe.json"                "california_housing_test.csv" 
[3] "california_housing_train.csv" "mnist_test.csv"              
[5] "mnist_train_small.csv"        "README.md"

In [ ]:
file.exists("/content/competition-data.csv")

[1] TRUE

In [ ]:
raw_data <- read_csv("/content/competition-data.csv")

Rows: 6000 Columns: 22
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (22): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
drop_na_raw_data <- raw_data %>% drop_na()
predictors_only <- select(drop_na_raw_data, -outcome)
# Change predictors_only from tible to data.frame as 
# preProcess only accepts dataframes
df_predictors_only <- data.frame(predictors_only) 
modifiedData_preprocessing_fit <- preProcess(
  df_predictors_only, 
  method = c("BoxCox", "center", "scale"))
modifiedData_preprocessing_fit

Created from 6000 samples and 21 variables

Pre-processing:
  - Box-Cox transformation (9)
  - centered (21)
  - ignored (0)
  - scaled (21)

Lambda estimates for Box-Cox transformation:
0.2, 0.4, 0.2, 0.1, 0, 0.1, -0.7, -0.2, 0.9

In [ ]:
transformed_predictors <- predict(
  modifiedData_preprocessing_fit, df_predictors_only)

preprocessed_data <- add_column(transformed_predictors, 
     outcome = drop_na_raw_data$outcome)

head(preprocessed_data)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,⋯,X13,X14,X15,X16,X17,X18,X19,X20,X21,outcome
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-0.4377536,-1.4569465,-1.836839,-1.9412301,-1.9778876,-0.6745550,-0.4918070,0.44376691,-1.502482578,-0.02049334,⋯,-0.2989366,-0.21246408,-0.5821616,-0.5500367,-0.8219316,-1.6356172,-1.4372363,1.6276059,1.2404332,96
2,-0.4377536,0.5445399,1.388494,0.7750767,0.6353550,0.3659072,1.0478299,0.05533532,1.251579923,0.01449814,⋯,-0.2989366,-0.21246408,-0.5968222,-0.5592355,1.2879519,0.7612187,0.9535142,0.7334529,0.5788107,76
3,-0.2457651,0.4733545,1.124330,0.6756062,0.7151469,0.3619054,-0.1913900,1.80327750,0.001382211,0.60935329,⋯,-0.2989366,0.01569037,-0.2750225,-0.2837313,0.4587064,0.9597572,0.2327625,-0.5523060,-0.5685329,80
4,-0.4377536,1.1970778,1.262294,2.5960461,2.2047848,2.8470094,0.5972045,-0.39783488,0.050149070,0.01449814,⋯,-0.2989366,-0.21246408,0.1948491,0.3974403,2.8182740,1.6891632,1.2907386,0.8326376,0.9350771,66
5,-0.4377536,0.5026858,-1.402303,-0.5452477,-1.4496359,-0.4344483,-0.3415985,-0.39783488,0.719159751,-0.31792092,⋯,-0.2989366,-0.21246408,-0.4502164,-0.4212534,-0.5610415,-0.4299666,0.8282980,0.6729451,1.0081393,94
6,-0.4377536,-1.9679910,-1.593643,-1.0103253,-1.0332844,-0.6745550,-0.4918070,-0.43020418,-1.189895253,-0.35291240,⋯,-0.2989366,-0.21246408,-0.5968222,-0.5592355,-0.8114260,-1.4637740,0.6752270,-0.2104281,0.9006598,96


In [ ]:
set.seed(100)
number_train <- floor(0.8 * nrow(preprocessed_data))
training_indicies <- sample(1:nrow(preprocessed_data), number_train)
train <- preprocessed_data[training_indicies,]
validate <- preprocessed_data[-training_indicies,]

folds <- createFolds(train$outcome, k = 5, returnTrain = TRUE)
ctrl <- trainControl(method = "cv", index = folds)

In [ ]:
trainXnnet <- train
validatennet <- validate

training_data_predicators <- trainXnnet %>% select(-outcome)
training_data_outcome <- trainXnnet %>% select(outcome)

In [ ]:
length(training_data_predicators)

[1] 21

In [ ]:
model <- keras_model_sequential() 
model %>% 
  layer_dense(units = 256, activation = "relu", input_shape = length(training_data_predicators)) %>% 
  layer_dropout(rate = 0.4) %>% 
  layer_dense(units = 256, activation = "relu") %>%
  layer_dropout(rate = 0.3) %>%
  layer_dense(units = 128, activation = "relu") %>%
  layer_dropout(rate = 0.2) %>%
  layer_dense(units = 1, activation = "linear")

In [ ]:
summary(model)

Model: "sequential_2"
________________________________________________________________________________
 Layer (type)                       Output Shape                    Param #     
 dense_11 (Dense)                   (None, 256)                     5632        
                                                                                
 dropout_8 (Dropout)                (None, 256)                     0           
                                                                                
 dense_10 (Dense)                   (None, 256)                     65792       
                                                                                
 dropout_7 (Dropout)                (None, 256)                     0           
                                                                                
 dense_9 (Dense)                    (None, 128)                     32896       
                                                                                
 dropo

In [ ]:
model %>% compile(
  loss= "mse",
  optimizer = "adam",
  metrics = c("mean_absolute_error")
)

In [ ]:
input_matrix_data <- as.matrix(training_data_predicators)
label_matrix_data <- as.matrix(training_data_outcome$outcome)

In [ ]:
model %>% fit(
    input_matrix_data, label_matrix_data,
    epochs = 140, batch_size = 50,

)

In [ ]:
train_scores <- model %>% evaluate(input_matrix_data, label_matrix_data, verbose = 0)

In [ ]:
train_scores

loss mean_absolute_error 
           6.018669            1.767053

In [ ]:
predicted_outcome_training = model %>% predict(input_matrix_data)
rmse(predicted_outcome_training, label_matrix_data)

[1] 2.453297

In [ ]:
validate_data_predicators <- validatennet %>% select(-outcome)
validate_data_outcome <- validatennet %>% select(outcome)
validate_matrix_data = as.matrix(validate_data_predicators)
validate_matrix_label = as.matrix(validate_data_outcome$outcome)

In [ ]:
predicted_outcome <- model %>% predict(validate_matrix_data)

In [ ]:
rmse(predicted_outcome, validate_matrix_label)

[1] 2.541914

In [ ]:
test_data <- read_csv("/content/competition-test-x-values.csv")

Rows: 2192 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (21): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
modified_test_data <- preProcess(
  data.frame(test_data), 
  method = c("BoxCox", "center", "scale"))
modified_test_data
transformed_test_data <- predict(
  modified_test_data, df_predictors_only)
testXnnet <- transformed_test_data
test_data_matrix <-  as.matrix(testXnnet)
predicted_test_data <- model %>% predict(test_data_matrix)
write.csv(predicted_test_data, file = "competition-test-outcome.csv")

Created from 2192 samples and 21 variables

Pre-processing:
  - Box-Cox transformation (10)
  - centered (21)
  - ignored (0)
  - scaled (21)

Lambda estimates for Box-Cox transformation:
0.2, 0.4, 0.2, 0.1, -0.1, -0.1, 0.1, -0.7, -0.2, 0.9

Warning message in predict.BoxCoxTrans(object$bc[[i]], newdata[, i]):
“newdata should have values > 0”
